# Build Resultant File
Builds the resultant files
This section of code takes the cleaned up layers and attirbues tables and pushes them together into one layer called the resultant. 
It then removes tidies up the resulant file by reparing the geometry, changing multipart to single part polygons and eliminating sliver and low area polygons by merging them with their largest neighbour. 
This code builds on and updates original code y Dr. Cosmin Mann.

In [ ]:
import arcpy
import time
Start = time.time()

#eliminator function to eliminate the sliver polygons 
def eliminator(fc,fcout):
    arcpy.MakeFeatureLayer_management(fc, "tempLayer")
    arcpy.SelectLayerByAttribute_management("tempLayer", "NEW_SELECTION", '"Shape_Area" < 1000')
    arcpy.Eliminate_management("tempLayer", fcout)
    arcpy.Delete_management("tempLayer")

arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\02_Data\Data_Prep_GoldenBear.gdb"

arcpy.Identity_analysis ("GoldenBear_AOI", "GoldenBear_BEC", "temp1","ALL", 0.1)
arcpy.Identity_analysis ("temp1", "GoldenBear_Caribou", "temp2","ALL", 0.1)
arcpy.Identity_analysis ("temp2", "GOLDENBEAR_F_Own", "temp3","ALL", 0.1)
arcpy.Identity_analysis ("temp3", "GOLDENBEAR_SOI", "temp4","ALL", 0.1)

arcpy.Identity_analysis ("temp4", "GoldenBear_TSA", "temp5","ALL", 0.1)
arcpy.Identity_analysis ("temp5", "GoldenBear_UWR", "temp6","ALL", 0.1)
arcpy.Identity_analysis ("temp6", "GoldenBear_VQO_", "temp7","ALL", 0.1)

arcpy.Identity_analysis ("temp7", "GoldenBear_LU_1", "temp8","ALL", 0.1)
arcpy.Identity_analysis ("temp8", "GoldenBear_Watersheds", "temp9","ALL", 0.1)
arcpy.Identity_analysis ("temp9", "GoldenBear_Lakes", "temp10","ALL", 0.1)
arcpy.Identity_analysis ("temp10", "GoldenBear_Rivers", "temp11","ALL", 0.1)
arcpy.Identity_analysis ("temp11", "GoldenBear_Wetlands", "temp12","ALL", 0.1)

eliminator("temp12","temp12e")
print("administrative_management completed: identifying inventory")

arcpy.Identity_analysis ("temp12e", "GoldenBear_R1_Poly", "temp13","ALL", 0.1)
arcpy.Identity_analysis ("temp13", "GoldenBear_Lakes_Buffer", "temp14","ALL", 0.1)
arcpy.Identity_analysis ("temp14", "GoldenBear_Rivers_Buffer", "temp15","ALL", 0.1)
arcpy.Identity_analysis ("temp15", "GoldenBear_Roads_Buffer", "temp16","ALL", 0.1)
arcpy.Identity_analysis ("temp16", "GoldenBear_streams_Buffer", "temp17","ALL", 0.1)
arcpy.Identity_analysis ("temp17", "GoldenBear_wetlands_Buffer", "temp18","ALL", 0.1)

print ("temp18 is the final feature dataset")
print ("repairing Geom, multi to sgl, eliminate more polys...")

arcpy.RepairGeometry_management ("temp18")
arcpy.MultipartToSinglepart_management("temp18","temp18_sgl")
eliminator("temp18_sgl","temp18_sgl_e1")
eliminator("temp18_sgl_e1","temp18_sgl_e2")
print ('It took ', round((time.time()-Start)/60,1), " minutes to run this script.")

## Manually double check the file, you may need to run the elimate tool again if there are still tiny polygons.
## The final manual steps are to delete the Temporary files that were created and to delete the FID_Temp... attributes that were created. These columns are not needed and add usless data. 
## Finally rename the data set and put into the resultant geodatabase. 

# Netdown Creation
This section of code delinates different areas of the land base. It creates three different column and sorts the data based on different values in the attribute tables.

In [ ]:
import arcpy
import time
Start = time.time()

arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\02_Data\Resultant_GoldenBear.gdb"

fc = "Resultand_GoldenBear_v2" 
try:
    arcpy.AddField_management(fc, "contclass", "TEXT", "", "", 1) 
except:
    pass 
try:
    arcpy.AddField_management(fc, "rollup", "TEXT", "", "", 50) 
except:
    pass 
try:
    arcpy.AddField_management(fc, "netdown", "TEXT", "", "", 50) 
except:
    pass 
try:
    arcpy.AddField_management(fc, "THLB_Area", "Double") 
except:
    pass

#dictionary that enables the use of fields name in the update cursor 
flist = arcpy.ListFields(fc)
fdic = {} 
fl = []
for f in flist:
    fdic[f.name] = flist.index(f) 
    fl.append(f.name)

with arcpy.da.UpdateCursor(fc, fl) as cursor:
    for row in cursor:
        row[fdic["contclass"]] = "C" 
        row[fdic["netdown"]] = "THLB" 
        row[fdic["rollup"]] = "THLB"
        row[fdic["THLB_Area"]] = row[fdic["Shape_Area"]]/10000
        
       ## Riparian from lake, wetland, river, and Stream buffers 
        if row[fdic["FID_GoldenBear_Lakes_Buffer"]] > 0:
            row[fdic["contclass"]] = "N" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "6_04_Lake_Buffer" 
            row[fdic["rollup"]] = "6_Riparian"
            
        if row[fdic["FID_GoldenBear_wetlands_Buffer"]] >0: 
            row[fdic["contclass"]] = "N" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "6_03_Wetland_Buffer" 
            row[fdic["rollup"]] = "6_Riparian"

        if row[fdic["FID_GoldenBear_Rivers_Buffer"]] >0: 
            row[fdic["contclass"]] = "N" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "6_02_River_Buffer" 
            row[fdic["rollup"]] = "6_Riparian"

        if row[fdic["FID_GoldenBear_streams_Buffer"]] >0: 
            row[fdic["contclass"]] = "N" 
            row[fdic["THLB_Area"]] = 0
            row[fdic["netdown"]] = "6_01_Stream_Buffer"
            row[fdic["rollup"]] = "6_Riparian"
            
        ## Non-productive/non-commercial stands
        if row[fdic["SPECIES_CD_1"]] not in [None, ""]:
            if row[fdic["NON_FOREST_DESCRIPTOR"]] not in [None, ""] :
                row[fdic["contclass"]] = "N" 
                row[fdic["THLB_Area"]] = 0
                row[fdic["netdown"]] = ("5_03_Non_Prod_"+ str(row[fdic["NON_FOREST_DESCRIPTOR"]]))[0:50] 
                row[fdic["rollup"]] = "5_Non_Productive"
            
        ##Environmentally Sensitive Areas (ESA)
        if row[fdic["MODIFYING_PROCESS"]] not in [None, "", "N"] :
            row[fdic["contclass"]] = "N" 
            row[fdic["THLB_Area"]] = 0
            row[fdic["netdown"]] = ("5_01_Sensitive_"+ str(row[fdic["MODIFYING_PROCESS"]]))[0:50] 
            row[fdic["rollup"]] = "5_Non_Productive"

        if row[fdic["HARVEST_DATE"]] is None and row[fdic["SITE_INDEX"]] is not None and row[fdic["SITE_INDEX"]] < 8:
        ## and row[fdic["FID_Results"]]<1 
            row[fdic["contclass"]] = "N"
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "4_01_Poor_sites" 
            row[fdic["rollup"]] = "4_Non_Commercial"
            
        ## Reserves+Habitat
        # current data set doesn't have no harvest all is conditional harvest
        #if row[fdic["FID_GoldenBear_UWR"]] >0 and row[fdic["TIMBER_HARVEST_CODE"]]=="NO HARVEST ZONE":
         #   row[fdic["contclass"]] = "N"
          #  row[fdic["THLB_Area"]] = 0
           # row[fdic["netdown"]] = ("3_04_HabitatUWR_"+ str(row[fdic["UWR_NUMBER"]]))[0:50] 
            #row[fdic["rollup"]] = "3_Reserves"

        #if row[fdic["FID_WHA_a_proj_clip"]] >0 and row[fdic["TIMBER_HARVEST_CODE_1"]]=="NO HARVEST ZONE": 
        #    row[fdic["contclass"]] = "N"
        #    row[fdic["THLB_Area"]] = 0
        #    row[fdic["netdown"]] = ("3_03_HabitatWHa_"+ str(row[fdic["COMMON_SPECIES_NAME"]]))[0:50] 
        #    row[fdic["rollup"]] = "3_Reserves"

        #if row[fdic["FID_WHA_p_proj_clip"]] >0 and row[fdic["TIMBER_HARVEST_CODE_12"]]=="NO HARVEST ZONE": 
        #    row[fdic["contclass"]] = "N"
        #    row[fdic["THLB_Area"]] = 0
        #    row[fdic["netdown"]] = ("3_03_HabitatWHp_"+ str(row[fdic["COMMON_SPECIES_NAME_1"]]))[0:50] 
        #    row[fdic["rollup"]] = "3_Reserves"

        #if row[fdic["FID_OGMA_proj_clip"]] >0: 
        #    row[fdic["contclass"]] = "N" 
        #    row[fdic["THLB_Area"]] = 0 
        #    row[fdic["netdown"]] = ("3_02_OGMA_")[0:50] 
        #    row[fdic["rollup"]] = "3_Reserves"
        
        ##Non Forest 
        #VRI Non_Forest

        if row[fdic["BCLCS_LEVEL_1"]] in [None, "", "U"]: 
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "2_08_Not_Typed" 
            row[fdic["rollup"]] = "2_Non_Forest"

        if row[fdic["BCLCS_LEVEL_1"]] == "N" and row[fdic["BCLCS_LEVEL_2"]] == "L" and (row[fdic["HARVEST_DATE"]] is None or row[fdic["HARVEST_DATE"]] < 1):
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["rollup"]] = "2_Non_Forest"
            if row[fdic["BCLCS_LEVEL_3"]] in ["A", "U"]:
                row[fdic["netdown"]] = "2_07_Non_vegetated_land" 
            else:
                row[fdic["netdown"]] = "2_02_Wetlands"

        if row[fdic["BCLCS_LEVEL_1"]] =="N" and row[fdic["BCLCS_LEVEL_2"]] =="W": 
            row[fdic["contclass"]] = "X"
            row[fdic["THLB_Area"]] = 0 
            row[fdic["rollup"]] = "2_Non_Forest"
            if row[fdic["BCLCS_LEVEL_3"]] =="A" and row[fdic["BCLCS_LEVEL_5"]] =="LA": 
                row[fdic["netdown"]] = "2_01_Lakes"
            else:
                row[fdic["netdown"]] = "2_03_Rivers"
            if row[fdic["BCLCS_LEVEL_3"]] =="W" and row[fdic["BCLCS_LEVEL_5"]] =="OC": 
                row[fdic["netdown"]] = "2_06_Ocean"
            else:
                row[fdic["netdown"]] = "2_02_Wetlands"
        
        if row[fdic["BCLCS_LEVEL_1"]] == "V" and row[fdic["BCLCS_LEVEL_2"]] == "N" and (row[fdic["HARVEST_DATE"]] is None or row[fdic["HARVEST_DATE"]] < 1):
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["rollup"]] = "2_Non_Forest"
            if row[fdic["BCLCS_LEVEL_3"]] in ["A", "U"]: 
                row[fdic["netdown"]] = "2_05_Vegetated_Not_Treed"
            else:
                row[fdic["netdown"]] = "2_02_Wetlands"
        
        if row[fdic["BCLCS_LEVEL_1"]] == "V" and row[fdic["BCLCS_LEVEL_2"]] == "T": 
            if row[fdic["BCLCS_LEVEL_3"]] == "W":
                row[fdic["contclass"]] = "X" 
                row[fdic["THLB_Area"]] = 0 
                row[fdic["netdown"]] = "2_02_Wetlands" 
                row[fdic["rollup"]] = "2_Non_Forest"

        if row[fdic["BCLCS_LEVEL_3"]] == "U" and row[fdic["SPECIES_CD_1"]] in [None, ""] and (row[fdic["HARVEST_DATE"]] is None or row[fdic["HARVEST_DATE"]] < 1):
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "2_08_Not_Typed" 
            row[fdic["rollup"]] = "2_Non_Forest"
            
        #Roads
        if row[fdic["FID_GoldenBear_Roads_Buffer"]] >0 : 
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "2_04_Roads" 
            row[fdic["rollup"]] = "2_Non_Forest"
            
        #Water
        if row[fdic["FID_GoldenBear_Wetlands"]]>0 : 
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "2_02_Wetlands" 
            row[fdic["rollup"]] = "2_Non_Forest"

        if row[fdic["FID_GoldenBear_Rivers"]]>0 : 
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "2_03_Rivers" 
            row[fdic["rollup"]] = "2_Non_Forest"

        if row[fdic["FID_GoldenBear_Lakes"]]>0 : 
            row[fdic["contclass"]] = "X"
            row[fdic["THLB_Area"]] = 0
            row[fdic["netdown"]] = "2_01_Lakes" 
            row[fdic["rollup"]] = "2_Non_Forest"
            
         ## Exclusions
        #if row[fdic["FID_Parks_Prv_proj_clip"]] >0:
        #    row[fdic["contclass"]] = "X" 
        #    row[fdic["THLB_Area"]] = 0
        #    row[fdic["netdown"]] = ("1_03_Park_"+ str(row[fdic["PROTECTED_LANDS_NAME"]]))[0:50] 
        #    row[fdic["rollup"]] = "1_Exclusions"

        #if row[fdic["FID_Parks_Fed_proj_clip"]] >0:
        #    row[fdic["contclass"]] = "X" 
        #    row[fdic["THLB_Area"]] = 0
        #    row[fdic["netdown"]] = ("1_03_Park_"+ str(row[fdic["ENGLISH_NAME_1"]]))[0:50] 
        #    row[fdic["rollup"]] = "1_Exclusions"
    
       # if row[fdic["FID_GoldenBear_ALC"]] >0: 
        #    row[fdic["contclass"]] = "X" 
         #   row[fdic["THLB_Area"]] = 0 
          #  row[fdic["netdown"]] = "1_02_AgricRsv" 
           # row[fdic["rollup"]] = "1_Exclusions"
    
        #if row[fdic["FID_IR_proj_clip"]] >0:
        #    row[fdic["contclass"]] = "X" 
        #    row[fdic["THLB_Area"]] = 0
        #    row[fdic["netdown"]] = ("1_01_IR_"+ str(row[fdic["ENGLISH_NAME"]]))[0:50] 
        #    row[fdic["rollup"]] = "1_Exclusions"

        cursor.updateRow(row)
print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")

# Sum the NetDown to put into table
## Rollup

In [ ]:
import csv
import arcpy
import time

Start = time.time()

# Set the workspace location:
arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\02_Data\Resultant_GoldenBear.gdb"

# Identify the feature class (table to work on)
fc = "Resultand_GoldenBear_v2"

# This bit of code enables you to reference the field name rather than referencing the row in the list by the index position (i.e. row[0], row[1])
flist = arcpy.ListFields(fc)
fdic = {}
fl = []
for f in flist:
    fdic[f.name] = flist.index(f)
    fl.append(f.name)

# This declares the dictionary that will be used to summarize the data
sumdict = {}

# Using the data access search cursor, go through all the rows. If the dictionary has a new key, add it to the list and add the area. Otherwise, if one is already added, just keep adding to the key
with arcpy.da.SearchCursor(fc, fl) as cursor:
    for row in cursor:
        if row[fdic["rollup"]] in sumdict:
            sumdict[row[fdic["rollup"]]] += row[fdic["Shape_Area"]] / 10000
        else:
            sumdict[row[fdic["rollup"]]] = row[fdic["Shape_Area"]] / 10000

# This section makes and sorts a list of the items in the sumdict dictionary and appends them into a CSV-like table
l = list(sumdict.items())
l.sort()
d = [["rollup", "Hectares"]]
for r in l:
    d.append(r)

# Specifies the output location of the CSV file
outfile = open(r"E:\Newmont Carbon Project\Data\Data BC\02_Data\GoldenBear_Resultant_Stats\SUM_rollup.csv", 'w', newline='', encoding='utf-8')
writer = csv.writer(outfile)
writer.writerows(d)
outfile.close()
print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")

## Netdown

In [ ]:
## summarize net down

import csv
import arcpy
import time

Start = time.time()

# Set the workspace location:
arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\02_Data\Resultant_GoldenBear.gdb"

# Identify the feature class (table to work on)
fc = "Resultand_GoldenBear_v2"

# This bit of code enables you to reference the field name rather than referencing the row in the list by the index position (i.e. row[0], row[1])
flist = arcpy.ListFields(fc)
fdic = {}
fl = []
for f in flist:
    fdic[f.name] = flist.index(f)
    fl.append(f.name)

# This declares the dictionary that will be used to summarize the data
sumdict = {}

# Using the data access search cursor, go through all the rows. If the dictionary has a new key, add it to the list and add the area. Otherwise, if one is already added, just keep adding to the key
with arcpy.da.SearchCursor(fc, fl) as cursor:
    for row in cursor:
        if row[fdic["netdown"]] in sumdict:
            sumdict[row[fdic["netdown"]]]=sumdict[row[fdic["netdown"]]]+row[fdic["Shape_Area"]]/10000
        else:
            sumdict[row[fdic["netdown"]]] = row[fdic["Shape_Area"]] / 10000

##This section makes and sorts a list of the items in the sumdict dictionary and appends them into a CSV like table
l = sumdict.items()
l = sorted(l)
d = [["netdown", "Hectares"]]
for r in l:
    d.append(r)

            
##Specifies the output location of the csv file
outfile = open(r"E:\Newmont Carbon Project\Data\Data BC\02_Data\GoldenBear_Resultant_Stats\SUM_netdown.csv", 'w', newline='', encoding='utf-8')

writer = csv.writer(outfile)
writer.writerows(d)
outfile.close()
print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")


# Add starting age column
This script adds fields and updates the stand age to year 2023. 

In [ ]:
import arcview
import arcpy
import time
Start = time.time()

arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\02_Data\Resultant_GoldenBear.gdb"

fc = "Resultand_GoldenBear_v2"

try:
    arcpy.AddField_management(fc, "Age_2023", "LONG")
except:
    pass

## short code to enable the use of field names
flist = arcpy.ListFields(fc)
fdic = {}
fl = []
for f in flist:
    fdic[f.name] = flist.index(f)
    fl.append(f.name)
    
#the update cursor
with arcpy.da.UpdateCursor(fc, fl) as cursor:
    for row in cursor:
        row[fdic["Age_2015"]]=None
        if row[fdic["FID_Results"]]>0:
            row[fdic["Age_2015"]]=2015-row[fdic["Dist_Year"]]
        elif row[fdic["PROJ_AGE_1"]] <> None:
            row[fdic["Age_2015"]] = row[fdic["PROJ_AGE_1"]]+1
        elif row[fdic["HARVEST_DATE"]]<> None:
            row[fdic["Age_2015"]]=2015-int(str(row[fdic["HARVEST_DATE"]])[:4])
        cursor.updateRow(row)
print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")

# Add AU fields
Adds AU filds and updates the AU based on the AU definitions from the Cassiar TSA.
The coastal/transitional groupings are the CWH, ESSF,
ICH, SBS, and MH zones, while the interior groupings are the BWBS and SWB zones.

Leading species present in area: AT, BL, HM, PL, SX

In [ ]:
import arcpy
import time

Start = time.time()

arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\02_Data\Resultant_GoldenBear.gdb"

fc = "Resultand_GoldenBear_v2"

try:
    arcpy.AddField_management(fc, "AU", "LONG")
except arcpy.ExecuteError:
    pass

# Short code to enable the use of field names
flist = arcpy.ListFields(fc)
fdic = {f.name: flist.index(f) for f in flist}
fl = list(fdic.keys())

# Define species lists
Aspen = ['AC', 'ACT', 'AT', 'EP', 'VB', 'MB']
Bal = ['B', 'BA', 'BG', 'BL']
Cedar = ['CW', 'YC']
Alder = ['D', 'DR']
DougFir = ['F', 'FD', 'FDC', 'FDI']
Hem = ['H', 'HM', 'HW']
Pine = ['PA', 'PL', 'PLC', 'PW', 'PLI', 'PY']
Spruce = ['S', 'SS', 'SW', 'SX', 'SE', 'SXW', 'SB']

# The update cursor
with arcpy.da.UpdateCursor(fc, fl) as cursor:
    for row in cursor:
        
        # Stands
        species_cd_1 = row[fdic["SPECIES_CD_1"]]
        site_index = row[fdic["SITE_INDEX"]]
        bec_zone_code = row[fdic["BEC_ZONE_CODE"]]
        herd_name = row[fdic["HERD_NAME"]] 

        # Put Decid into its own group
        if species_cd_1 in Aspen:
            row[fdic["AU"]] = 33

        # Coastal-Spruce
        elif species_cd_1 in Spruce and bec_zone_code in ["CWH", "ESSF", "ICH", "SBS", "MH"]:
            if site_index is None or 17.1 <= site_index <= 25.0:
                row[fdic["AU"]] = 2
            elif site_index >= 25.1:
                row[fdic["AU"]] = 1
            elif 0.1 < site_index < 17.1:
                row[fdic["AU"]] = 3

        # Interior-Spruce
        elif species_cd_1 in Spruce and bec_zone_code in ["BWBS", "SWB"]:
            if site_index is None or 17.1 <= site_index <= 25.0:
                row[fdic["AU"]] = 5
            elif site_index >= 25.1:
                row[fdic["AU"]] = 4
            elif 0.1 < site_index < 17.1:
                row[fdic["AU"]] = 6

        # Alpine Spruce
        elif species_cd_1 in Spruce and bec_zone_code in ["BAFA", "CMA"]:
            row[fdic["AU"]] = 7

        # Coastal Pine
        elif species_cd_1 in Pine and bec_zone_code in ["CWH", "ESSF", "ICH", "SBS", "MH"]:
            if site_index is None or 14.6 <= site_index <= 19.0:
                row[fdic["AU"]] = 9
            elif site_index >= 19.1:
                row[fdic["AU"]] = 8
            elif 0.1 < site_index < 14.6:
                row[fdic["AU"]] = 10

        # Interior Pine
        elif species_cd_1 in Pine and bec_zone_code in ["BWBS", "SWB"]:
            if site_index is None or 14.6 <= site_index <= 19.0:
                row[fdic["AU"]] = 12
            elif site_index >= 19.1:
                row[fdic["AU"]] = 11
            elif 0.1 < site_index < 14.6:
                row[fdic["AU"]] = 13

        # Alpine Pine
        elif species_cd_1 in Pine and bec_zone_code in ["BAFA", "CMA"]:
            if site_index is None or 14.6 <= site_index <= 19.0:
                row[fdic["AU"]] = 15
            elif site_index >= 19.1:
                row[fdic["AU"]] = 14
            elif 0.1 < site_index < 14.6:
                row[fdic["AU"]] = 16

   # Coastal Balsam
        if species_cd_1 in Bal and bec_zone_code in ["CWH", "ESSF", "ICH", "SBS", "MH"]:
            if site_index is None or 13.1 <= site_index <= 16.0:
                row[fdic["AU"]] = 18
            elif site_index >= 16.1:
                row[fdic["AU"]] = 17
            elif 0.1 < site_index < 13.1:
                row[fdic["AU"]] = 19

         # Interior Balsam
        elif species_cd_1 in Bal and bec_zone_code in ["BWBS", "SWB"]:
            if site_index is None or 13.1 <= site_index <= 16.0:
                row[fdic["AU"]] = 21
            elif site_index >= 16.1:
                row[fdic["AU"]] = 20
            elif 0.1 < site_index < 13.1:
                row[fdic["AU"]] = 22
                
        # Alpine Balsam
        elif species_cd_1 in Bal and bec_zone_code in ["BAFA", "CMA"]:
            row[fdic["AU"]] = 23

   # Coastal Hemlock
        if species_cd_1 in Hem and bec_zone_code in ["CWH", "ESSF", "ICH", "SBS", "MH"]:
            if site_index is None or 14.1 <= site_index <= 17.0:
                row[fdic["AU"]] = 25
            elif site_index >= 17.1:
                row[fdic["AU"]] = 24
            elif 0.1 < site_index < 14.1:
                row[fdic["AU"]] = 26

         # Interior Hemlock
        elif species_cd_1 in Hem and bec_zone_code in ["BWBS", "SWB"]:
            if site_index is None or 14.1 <= site_index <= 17.0:
                row[fdic["AU"]] = 28
            elif site_index >= 17.1:
                row[fdic["AU"]] = 27
            elif 0.1 < site_index < 14.1:
                row[fdic["AU"]] = 29
                
        # Alpine Hemlock
        elif species_cd_1 in Hem and bec_zone_code in ["BAFA", "CMA"]:
            if site_index is None or 14.1 <= site_index <= 17.0:
                row[fdic["AU"]] = 30
            elif 0.1 < site_index < 14.1:
                row[fdic["AU"]] = 31

#Caribou stands (overwrites the previous stands if these stands are in the Caribou zone)
        if herd_name in ["Level-Kawdy"]:
            row[fdic["AU"]]=32

        cursor.updateRow(row)

print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")

 
                
# #Caribou stands (overwrites the previous stands if these stands are in the Caribou zone)
#         if row[fdic["HERD_NAME"]]== "Itcha-Ilgachuz":
#             if row[fdic["FID_Results"]]>0:
#                 row[fdic["AU"]]=7
#             else:
#                 row[fdic["AU"]]=6

# #MDWR stands (overwrites the previous non-MPB and Caribou stands if in MDWR zone)
#         if row[fdic["FID_UWR_proj_clip"]]>1:
#             if row[fdic["SPECIES_CD_1"]] in DougFir and row[fdic["SPECIES_PCT_1"]] >=40:
#                 if row[fdic["BEC_ZONE_CODE"]] in ["IDF", "SBPS"]:
#                     if row[fdic["HARVEST_DATE"]] not in ['',None] or row[fdic["OPENING_IND"]]== "Y" or row[fdic["FID_Results"]]>0:
#                         row[fdic["AU"]]=2
#                     else:
#                         row[fdic["AU"]]=3
#                 if row[fdic["BEC_ZONE_CODE"]] in ["SBS", "ICH"]:
#                     row[fdic["AU"]]=4
#                 else:
#                     row[fdic["AU"]]=1
#             else:
#                 row[fdic["AU"]]=5

# #recent harvest blocks with no species info assumed PL_med
# if row[fdic["AU"]]==None and (row[fdic["HARVEST_DATE"]] not in ['',None] or row[fdic["FID_Results"]]>0):
# row[fdic["AU"]]=17
